In [1]:
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory
from tensorflow import keras
from keras import layers
from keras import models
import tensorflow as tf
from keras.preprocessing import image
tf.config.list_physical_devices('GPU')

2024-03-08 22:19:46.648400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 22:19:46.648449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 22:19:46.650379: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 22:19:46.659176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 22:19:47.681499: W tensorflow/compiler/tf2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
train_dir = '../Generator/dataset/train'
test_dir = '../Generator/dataset/test'

train_square_dir = train_dir + '/square'
train_circle_dir = train_dir + '/circle'
train_none_dir = train_dir + '/none'
train_circle_square_dir = train_dir + '/square_circle'
test_circle_square_dir = test_dir + '/square_and_circle'

In [3]:
IMG_SIZE = 500
train_datagen = ImageDataGenerator(
    #rescale=1./255,  # Normalize pixel values
    #rotation_range=20,  # Randomly rotate images up to 20 degrees
    #width_shift_range=0.2,  # Shift images horizontally up to 20%
    #height_shift_range=0.2,  # Shift images vertically up to 20%
    #shear_range=0.2,  # Shear images randomly
    #zoom_range=0.2,  # Zoom images randomly
    #horizontal_flip=True  # Flip images horizontally
)

train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to training directory
    target_size=(IMG_SIZE, IMG_SIZE),  # Set your image dimensions
    batch_size=32,  # Adjust batch size as needed
    class_mode='categorical'  # One-hot encode labels for multi-class classification
)

Found 3000 images belonging to 4 classes.


In [4]:
train_dataset = image_dataset_from_directory(train_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=32)
test_dataset = image_dataset_from_directory(test_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=32)

Found 3000 files belonging to 4 classes.


2024-03-08 22:19:54.973483: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-08 22:19:54.973981: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-08 22:19:54.974149: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 1000 files belonging to 1 classes.


In [5]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [6]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [7]:
history = model.fit(train_generator, epochs=10, batch_size=32)

Epoch 1/10


2024-03-08 22:20:05.714613: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-08 22:20:05.945110: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-08 22:20:12.217781: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f4d9c984df0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-08 22:20:12.217826: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 with Max-Q Design, Compute Capability 6.1
2024-03-08 22:20:12.225615: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709936412.280743  199528 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


94/94 [==============================] - 63s 477ms/step - loss: 1.4040 - acc: 0.2513
Epoch 2/10
21/94 [=====>........................] - ETA: 25s - loss: 1.3866 - acc: 0.2664

In [ ]:
img = np.array([])
for data_batch, label in test_dataset.take(1):
        image = data_batch[1].numpy().astype("uint8")
        img = np.append(img, image)
        plt.imshow(image)
        plt.show()
        break
img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
img = np.expand_dims(img, axis=0)
print(img.shape)
print(model.predict(img).round())


In [ ]:
# model.save('models/04_CNN_02_CatsAndDogs_01_cnn_from_scratch.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
img = tf.keras.preprocessing.image.load_img('cats_and_dogs_small/validation/dogs/dog.1010.jpg', target_size=(150, 150), interpolation='bilinear')

In [ ]:
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
print(img_array.shape)
result = model.predict(img_array)
print("Result: ", result.round())